<font size="+1" color="RED">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms :**
Antoine Toullalan,
Rosa Mendas

## Partie 1 - Description du problème

Nous avons vu que les percepeptrons ne permettaient pas d'avoir une bonne prédiction du nombre de téléchargements d'une app. On va donc essayer de prédire le nombre de téléchargement d'une app grâce aux arbres de décisions. 

## Partie 2 - Modèle

On importe d'abord les données "nettoyées" de googleplaystoreCLEANpb2.csv, on importe aussi les fonctions de constructions d'arbres de décision du fichier "Arbre".
Nous allons créer un arbre de décision qui nous donne dans quelle catégorie se situe une app au niveau du nombre de téléchargements (les catégories sont : de 0 à 100, de 100 à 1000, de 1000 à 10000... jusqu'à 1 000 000 000), il y a donc 9 catégories.

## Partie 3 - Code

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
import graphviz as gv

from iads import Classifiers as classif
from statistics import mean
%matplotlib inline

import sys
sys.path.append('../')  

import iads as iads

# importation de Classifiers
from iads import Arbres as arbres
from iads import utils as ut

data = pd.read_csv("GoogleApps/googleplaystoreCLEANpb2.csv",encoding='utf-8')
data=data.drop("Unnamed: 0",axis=1)
data=data.drop("Rating",axis=1)
#on supprime la colonne sur le nom des app et Android Ver car il est assez difficile de tirer une information exploitable de ses valeurs
data=data.drop("App",axis=1)
data=data.drop("Android Ver",axis=1)
print(data.columns)

Index(['Category', 'Size', 'Installs', 'Price', 'Content Rating'], dtype='object')


Nous avons un problème lors de la construction de l'arbre de décision : il semble qu'il ya trop d'attributs donc nous atteignons la profondeur maximale de récursion, il faut donc réduire le nombre d'attributs.

In [23]:
def leave_one_out(C, X,Y,nb_tests):
    acc=0
    for _ in range(nb_tests):
        i=random.randrange(len(X))
        
        one_out=X[i]
        label_one_out=Y[i]
        
        X2=np.concatenate((X[:i],X[i+1:]))
        Y2=np.concatenate((Y[:i],Y[i+1:]))
        C.train(X2,Y2)
        if(C.predict(one_out)==label_one_out):
            acc+=1
    return acc/nb_tests

In [24]:
data_noms = [nom for nom in data.columns if nom != 'Installs']
data_desc = np.array(data[data_noms])
data_label = np.array(data['Installs'])

In [25]:
arbre_data = arbres.ClassifierArbreDecision(len(data_noms), 0.1, data_noms)
print("Accuracy avec leave_one_out : ",leave_one_out(arbre_data,data_desc,data_label,120))

	*** Warning: attribut  att_0  -> Valeur inconnue:  PHOTOGRAPHY
	*** Warning: attribut  att_0  -> Valeur inconnue:  SOCIAL
	*** Warning: attribut  att_0  -> Valeur inconnue:  SPORTS
Accuracy avec leave_one_out :  0.075


On voit que les résultats sont assez faibles (16% de prédictions correctes environ pour 10 valeurs de labels possibles) lorsqu'on utilise les données de "googleplaystoreCLEANpb2.csv". On suppose que le problème vient du fait que l'arbrea comme feuille des labels qui ont 10 valeurs différentes, et il est compliqué de créer un tel arbre. Il serait plus simple de créer l'arbre qui prédit le nombre de téléchargements si :
les labels ne peuvaient avoir que 2 valeurs différentes: on catégorise donc la valeur de la colonne "Installs" en 2 labels: -1 et 1, la valeur de la coupure est la médiane des valeurs de la colonne "Installs".

De plus on va catégoriser les valeurs des attributs en moins de valeurs pour faciliter l'apprentissage -> il ya notamment de nombreuses valeurs possibles de "size", on pourrait améliorer l'apprentissage de l'arbre en re-catégorisant cette colonne.

In [26]:
data_label2=data_label.copy()
data_label2.sort()
valeur_coupure=data_label2[int(len(data_label2)/2)]
print(valeur_coupure)

5


Donc la médiane des labels est 5, on catégorise de la façon suivante les labels:
si le label<5 -> label=-1 sinon label=1

In [27]:
data_label[data_label<=valeur_coupure]=-1
data_label[data_label>valeur_coupure]=1
print(data_label)

[-1 -1  1 ... -1 -1  1]


In [30]:
data_noms = [nom for nom in data.columns if nom != 'Installs']
#dans data_desc, on aura les colonnes size et price modifiées
data_desc = np.array(data[data_noms])
#on a déjà modifié data_label donc on ne le reconstruit pas
arbre_data = arbres.ClassifierArbreDecision(len(data_noms), 0.1, data_noms)
print("Accuracy avec leave_one_out : ",leave_one_out(arbre_data,data_desc,data_label,100))

Accuracy avec leave_one_out :  0.6


## Partie 4 - Protocole expérimental

Nous avons utilisé les fonctions développée au tme6, ainsi pour chaque attribut, on regarde le gain d'information. On choisit l'attribut qui maximise le gain d'information et pour chacune des valeurs que prend l'attribut on crée un noeud qui devient un fils du noeud initial (ce noeud est la racine au départ).
Lorsque le désordre à un noeud est suffisament faible, ce noeud est une feuille qui prend la valeur de a classe majoritaire.

Il faut aussi que les données soient exploitables par l'arbre, c'est pourquoi on re-catégorise le label en 2 catégories, on re-catégorise d'autres attributs (Price,Size) en 2 catégories pour favoriser l'apprentissage.

## Partie 5 - Résultats

Dans le 2eme arbre qu'on construit on arrive à avoir 63% de prédictions correctes pour prédire si une app aura plus ou moins de 10^6(=1 million) de téléchargements car la valeur de coupure est 5->les catégories sont 0 à 10^6(non inclus) et 10^6 (inclus) à 10^9 
Ce n'est pas très performant car comme on a 2 catégories, un classifieur aléatoire aurait une accuracy de 50%.
Donc notre classifieur est un peu plus performant qu'un classifieur aléatoire...

## Partie 6 - Analyse

On voit que notre classifieur donnent des résultat plus performants que celui d'un classifieur aléatoire, mais l'Accuracy de ce classifieur reste assez faible (un peu plus de 60%). 
On remarque que lorsqu'on rajoute la colonne "Rating", l'accuracy monte à 75% mais on était plus alors dans le modèle de notre développeur qui n'a accès qu'aux données de l'app avant qu'il la mette sur le GooglePlayStore. J'ai eu alors l'idée de remplacer les valeurs de Rating par la prédiction d'un classifieur car on a vu que le classifieur avec un perceptron à une assez bonne performance pour prédire le rating d'une app (env 60%), mais alors l'accuracy de l'arbre à baissé à 54%...

J'ai aussi catégorisé la classe "Size" en 2 catégories mais on est tombé à une accuracy de 41%.

Donc avec un arbre de classification qui prédit si l'app aura plus ou moins de 1 000 000 de téléchargements, on arrive à avoir une performance légèrement supérieur à celle d'un classifieur aléatoire mais cela reste assez faible car le succès d'une app dépend aussi (et surtout) de données plus subjectives que celles qu'on a.